# Change Detection with Unbihexium

[![CI](https://github.com/unbihexium-oss/unbihexium/workflows/CI/badge.svg)](https://github.com/unbihexium-oss/unbihexium/actions)
[![PyPI](https://img.shields.io/pypi/v/unbihexium.svg)](https://pypi.org/project/unbihexium/)
[![License](https://img.shields.io/badge/license-Apache--2.0-blue.svg)](LICENSE.txt)

**Author**: Unbihexium OSS Foundation  
**Version**: 1.0.0

---

## Purpose

Detect changes between bi-temporal satellite imagery using:
- Siamese neural networks
- Image differencing
- Deep change vector analysis

## Mathematical Foundation

$$\Delta I = |I_{t_1} - I_{t_2}|$$

$$P(\text{change}) = \sigma(f_{siamese}(I_{t_1}, I_{t_2}))$$

| Approach | Pros | Cons |
|----------|------|------|
| Differencing | Simple | Sensitive to noise |
| Siamese | Robust | Requires training |
| CVA | Multi-band | Complex |

In [ ]:
import numpy as np
from pathlib import Path

try:
    import onnxruntime as ort
    print(f"ONNX Runtime: {ort.__version__}")
except ImportError:
    print("Install: pip install onnxruntime")

## Simple Image Differencing

In [ ]:
def image_difference(img1: np.ndarray, img2: np.ndarray, threshold: float = 0.3) -> np.ndarray:
    """Compute change mask using absolute difference."""
    diff = np.abs(img1.astype(float) - img2.astype(float))
    diff_normalized = diff / (diff.max() + 1e-10)
    return diff_normalized > threshold

# Simulate bi-temporal images
np.random.seed(42)
t1 = np.random.rand(64, 64, 3).astype(np.float32)
t2 = t1.copy()

# Add synthetic change area
t2[20:40, 20:40, :] += 0.5
t2 = np.clip(t2, 0, 1)

# Detect changes
change_mask = image_difference(t1.mean(axis=-1), t2.mean(axis=-1))
print(f"Changed pixels: {change_mask.sum()} ({change_mask.mean()*100:.1f}%)")

## Siamese Network Approach

In [ ]:
# Load Siamese change detector
model_path = Path("../model_zoo/assets/tiny/change_detector_tiny/model.onnx")

if model_path.exists():
    session = ort.InferenceSession(str(model_path))
    
    # Prepare input (concatenate two images along channel dimension)
    t1_tensor = np.transpose(t1[:32, :32, :], (2, 0, 1))[np.newaxis, :].astype(np.float32)
    t2_tensor = np.transpose(t2[:32, :32, :], (2, 0, 1))[np.newaxis, :].astype(np.float32)
    combined = np.concatenate([t1_tensor, t2_tensor], axis=1)
    
    print(f"Input shape: {combined.shape}")
    
    # Run inference
    outputs = session.run(None, {"input": combined})
    print(f"Output shape: {outputs[0].shape}")
else:
    print(f"Model not found at {model_path}")

## Applications

| Application | Use Case |
|-------------|----------|
| Urban Growth | City expansion monitoring |
| Deforestation | Forest loss detection |
| Disaster | Damage assessment |
| Agriculture | Field changes |

---

**Copyright 2025 Unbihexium OSS Foundation. Apache-2.0 License.**